In [53]:
import pandas as pd
from datetime import datetime
import numpy as np

# STATION 

Description: This script takes in a file named Station.csv(default) , recognizes its information and interprets the columns. Finally prints and save a Dat format file with the predefined tables for further processing

Station Data Conversion REFERENCE : 

ORDER : Padded with incremental index numbering

Key : Column "Name" in Station.csv  . Erased blank spaces.('Example_ _')

Name : Column "Desc" in Station.csv . Erased blank spaces.('Example_ _')

AOR : Column Zones in Station.csv. 

The .str.rstrip() method is widely used since many fields (almost all of them) have their value ending with two blank spaces.

In [54]:
#Input Filename (default = Station.csv)
InputStationFile = "Station.csv"

In [55]:
df = pd.read_csv(InputStationFile, skipinitialspace=True)
df.columns = df.columns.str.strip()

df_output = pd.DataFrame()
df_output['Order'] = range(1, len(df)+1) # Order column was not  specified in documentation. So in default is a unique incremental index.
df_output['Key'] = '"'+ df['Name'].str.rstrip() + '"'  
df_output['Name'] = '"' + df['Desc'].str.rstrip() + '"' 
df_output['AOR'] = df['Zones']

#sorting
df_output = df_output[['Order', 'Key', 'Name', 'AOR']]
df_station = df_output

#verification
print(df_station)

    Order     Key                        Name                AOR
0       1    "DI"     "DR INLET DISTRIBUTION"                3  
1       2    "BY"      "BRICKYARD SUBSTATION"                4  
2       3    "WS"  "WORTHINGTON SPRINGS DIST"                5  
3       4    "WA"          "WALDO SUBSTATION"                1  
4       5    "AB"           "ASTOR SWITCHING"                1  
..    ...     ...                         ...                ...
70     71  "SYS$"       "SYTEM PSUEDO POINTS"  1 2 3 4 5 6 7 8  
71     72    "BL"          "BLAND SUBSTATION"                5  
72     73    "WO"    "WATER OAK DISTRIBUTION"                3  
73     74    "AS"        "ASTOR DISTRIBUTION"                7  
74     75  "ODA$"         "ODA PSEUDO ANALOG"  1 2 3 4 5 6 7 8  

[75 rows x 4 columns]


# Select Output name: (default: StationOutput.dat)

In [56]:
#Output:
stationfilename = "Station99.dat"

In [57]:
now = datetime.now().strftime("%d/%m/%Y %H:%M:%S")
with open(stationfilename, 'w') as f:
    f.write("*\n")
    f.write("****************************************************************\n")
    f.write(f"*  Creation Date/Time:  {now}\n")
    f.write("****************************************************************\n")
    f.write("*             Order  Key             Name                AOR\n")
    f.write("*             ----  -----   --------------------------   ---\n")
    f.write("\t2\tSTATION\t0\t3\t4\t13\n")
    f.write("*---------------------------------------------------------------\n")

    for index, row in df_station.iterrows():
        f.write("{}\t{}\t{}\t{}\t{}\t{}\n".format
                ("", row['Order'], row['Order'], row['Key'], row['Name'], row['AOR']))
    f.write(" 0")  

# END STATION

--------------------------------------------------

# STATUS

Description: This script takes in a file named Status.csv(default) , recognizes its information and interprets the columns. Finally prints and save a Dat format file with the predefined tables for further processing

Station Data Conversion REFERENCE : 

(1)Type : ( 
       
            If column "Telem_A" in status.csv = empty. C_IND.   Type = 5

            If column "Telem_A" in status.csv = any number. T_IND.  Type = 1
            
            If column "Open_B" in status.csv = 12.  T_I&C.     Type = 2  ) 
                Criteria: only 42 rows (0,49%) has Open_B= 12.
                        so its gonna be Type = 2 regardless the value of Telem_A, 
                        giving priority to Open_B.
                        (In evey case of Open_B=12, Telem_A showed a number also)
              
              Edited:  IF column Stn = 054 (PS. PSEUDO POINTS) , M_IND. Type = 8 

(3)Key : Format XXYYYZZZ
                         XX =   If type (descripted before) = 1 , XX = 01
                                If type (descripted before) = 2 , XX = 01
                                If type (descripted before) = 5 , XX = 02
                                If type (descripted before) = 8 , XX = 12
                         YYY = Stn (Station Order number.)(descripted in this Markdown)
                         ZZZ = Incremental number per station

(4)Name : Column "Name" + column "Desc" in Status.csv. Replace the commas (,) with blank spaces.

      *****
       Updated : 2019-03-06 . (4) Name : Column "Desc" in Status.csv . If not defined will be set to Key 

(5) Stn : (XXX) -> The first characters of the "Name" column in the current dataset, before the comma (,), reference the value of the KEY column in the previous dataset (Station). This value is based on the Order column. This number should be expressed in three digits (e.g., 38 = 038).

(10) Aor : Column "Zones" in Status.csv

(19) pState : Column "Presuffx" in Status.csv

(49) Norm : Column "Normal_State" in Status.csv

(29) AlarmGroup: Will be set to 1 unless defined in mapping document

(41) ICAddress : Refer to documentation. This script leaves it in blank

In [61]:
#Input filename(default: "Status.csv")
InputStatusFile = "Status.csv"

In [62]:
status_df = pd.read_csv(InputStatusFile)
df_status = pd.DataFrame()

In [63]:
df_status['record'] = range(1, len(status_df)+1) 
df_status['OrderNo'] = range(1, len(status_df)+1) 


df_status['Type'] = np.where(status_df['Open_B  '] == '12  ', 2, np.where(status_df['Telem_A  '].replace('  ', np.nan).notna(), 1, 5))
#df_status['Name'] = status_df['Name  '].str.replace(',',' ') + " " + status_df['Desc  ']
#df_status['Name'] = df_status['Name'].str.rstrip()
df_status['Name'] = status_df['Desc  ']
df_status['AOR'] = status_df['Zones  '].str.rstrip()
df_status['pState'] = status_df['PreSuffx  ']
df_status['Norm'] = status_df['Normal_State  ']
df_status['AlarmGroup'] = 1
df_status['ICAddress'] = np.nan

In [64]:
df_status

,record,OrderNo,Type,Name,AOR,pState,Norm,AlarmGroup,ICAddress
0,1,1,1,MOBIL TX HOT OIL TEMP,4,8,0,1,NaN
1,2,2,1,PH DIFF RELAY DISABLE,5,5,0,1,NaN
2,3,3,1,BW 2100-1 ALARM COM 9,4,26,0,1,NaN
3,4,4,5,SB PSEUDO BREAKER,1,15,0,1,NaN
4,5,5,1,YARD BOX 2505 SELF CHECK,5,26,0,1,NaN
...,...,...,...,...,...,...,...,...,...
8459,8460,8460,5,PSUEDO SWITCH,1 2 3 4 5 6 7 8,15,0,1,NaN
8460,8461,8461,5,FL LOW SD BUS TIE 3T12,1 2 3 4 5 6 7 8,15,0,1,NaN
8461,8462,8462,5,FL LOW SD BUS TIE 3T21,1 2 3 4 5 6 7 8,15,0,1,NaN
8462,8463,8463,5,FL XFR BUS TIE 3X12,1 2 3 4 5 6 7 8,15,0,1,NaN


# Stn :

This code strips the Name column in status and search the matching row Key in Station Dataframe
then, it reads the Order column value and assign this to the Stn column

In [10]:
status_df['Key'] = status_df['Name  '].str.split(',').str[0].str.strip()  # make sure no blank spaces are in the begin & end of the name
df_station['Key'] = df_station['Key'].str.replace('"', '')
stn_values = []  #list for store stn values

In [11]:
for i in range(len(status_df)):
    matching_row = df_station[df_station['Key'] == status_df.loc[i, 'Key']]
    if not matching_row.empty:
        stn_values.append(f"{matching_row['Order'].values[0]:03}")
    else:
        stn_values.append(np.nan)
df_status['Stn'] = stn_values

In [12]:
df_status

,record,OrderNo,Type,Name,AOR,pState,Norm,AlarmGroup,ICAddress,Stn
0,1,1,1,OP MTXTOT MOBIL TX HOT OIL TEMP,4,8,0,1,NaN,016
1,2,2,1,PH DRD PH DIFF RELAY DISABLE,5,5,0,1,NaN,025
2,3,3,1,BW 35C109 BW 2100-1 ALARM COM 9,4,26,0,1,NaN,015
3,4,4,5,PS S8W219 SB PSEUDO BREAKER,1,15,0,1,NaN,055
4,5,5,1,PH 25YBSC YARD BOX 2505 SELF CHECK,5,26,0,1,NaN,025
...,...,...,...,...,...,...,...,...,...,...
8459,8460,8460,5,PS FL3103 PSUEDO SWITCH,1 2 3 4 5 6 7 8,15,0,1,NaN,055
8460,8461,8461,5,PS FL3T12 FL LOW SD BUS TIE 3T12,1 2 3 4 5 6 7 8,15,0,1,NaN,055
8461,8462,8462,5,PS FL3T21 FL LOW SD BUS TIE 3T21,1 2 3 4 5 6 7 8,15,0,1,NaN,055
8462,8463,8463,5,PS FL3X12 FL XFR BUS TIE 3X12,1 2 3 4 5 6 7 8,15,0,1,NaN,055


Replacement case: Pseudo Points.
IF stn = 054 (PS. Pseudo points.) Set type to 8 .   And then set XX Value to 12

In [13]:
df_status.loc[df_status['Name'].str.startswith("PS"), 'Type'] = 8

for i in range(len(df_status)):
    if df_status.loc[i, 'Stn'] == '054':
        key_to_search = df_status.loc[i, 'Name'][3:5]

        # Buscar valor 
        matching_row = df_station[df_station['Key'] == key_to_search]

        # != una fila correspondiente, 'Stn' en 'df_status'
        if not matching_row.empty:
            df_status.loc[i, 'Stn'] = f"{matching_row['Order'].values[0]:03}"

# KEY

In [14]:
key_values = []
xx_yyy_counters = {}  # creating a dict to store and count every YYY. It's used for a incremental ZZZ

In [15]:
for i in range(len(df_status)):
    
    if df_status.loc[i, 'Type'] == 1:
        xx = '01'
    elif df_status.loc[i, 'Type'] == 2:
        xx = '01'
    elif df_status.loc[i, 'Type'] == 5:
        xx = '05'
    elif df_status.loc[i, 'Type'] == 8:
        xx = '12'
    else:
        xx = '99'  # ERROR CASE . In case of being unable to find a coincidence.

    yyy = df_status.loc[i, 'Stn']

    # combination
    key = xx + yyy

    # if key exists in counters, then add +1 , else, initialites it.
    if key in xx_yyy_counters:
        xx_yyy_counters[key] += 1
    else:
        xx_yyy_counters[key] = 1

    #   Current Value of ZZZ . 
    zz = f"{xx_yyy_counters[key]:03}"  #03 (Default). the number of Z in the format (default XX YYY ZZZ (3 Z))

    key_values.append(xx + yyy + zz)


In [16]:
df_status['Key'] = key_values

In [17]:
#obtaining and showing the counting of Types.
type_counts = df_status['Type'].value_counts()
print(type_counts)

Type
1    6437
8    1640
5     345
2      42
Name: count, dtype: int64


In [18]:
new_order = ['record', 'OrderNo','Type','Key','Name','Stn','AOR','pState','Norm','AlarmGroup','ICAddress']
df_status = df_status[new_order]

In [19]:
df_status

,record,OrderNo,Type,Key,Name,Stn,AOR,pState,Norm,AlarmGroup,ICAddress
0,1,1,1,01016001,OP MTXTOT MOBIL TX HOT OIL TEMP,016,4,8,0,1,NaN
1,2,2,1,01025001,PH DRD PH DIFF RELAY DISABLE,025,5,5,0,1,NaN
2,3,3,1,01015001,BW 35C109 BW 2100-1 ALARM COM 9,015,4,26,0,1,NaN
3,4,4,8,12055001,PS S8W219 SB PSEUDO BREAKER,055,1,15,0,1,NaN
4,5,5,1,01025002,PH 25YBSC YARD BOX 2505 SELF CHECK,025,5,26,0,1,NaN
...,...,...,...,...,...,...,...,...,...,...,...
8459,8460,8460,8,120551636,PS FL3103 PSUEDO SWITCH,055,1 2 3 4 5 6 7 8,15,0,1,NaN
8460,8461,8461,8,120551637,PS FL3T12 FL LOW SD BUS TIE 3T12,055,1 2 3 4 5 6 7 8,15,0,1,NaN
8461,8462,8462,8,120551638,PS FL3T21 FL LOW SD BUS TIE 3T21,055,1 2 3 4 5 6 7 8,15,0,1,NaN
8462,8463,8463,8,120551639,PS FL3X12 FL XFR BUS TIE 3X12,055,1 2 3 4 5 6 7 8,15,0,1,NaN


In [20]:
df_status['Key'] = '"'+ df_status['Key'].str.rstrip() + '"'

In [21]:
df_status['Name'] = '"'+ df_status['Name'].str.rstrip() + '"'

In [22]:
df_status

,record,OrderNo,Type,Key,Name,Stn,AOR,pState,Norm,AlarmGroup,ICAddress
0,1,1,1,"""01016001""","""OP MTXTOT MOBIL TX HOT OIL TEMP""",016,4,8,0,1,NaN
1,2,2,1,"""01025001""","""PH DRD PH DIFF RELAY DISABLE""",025,5,5,0,1,NaN
2,3,3,1,"""01015001""","""BW 35C109 BW 2100-1 ALARM COM 9""",015,4,26,0,1,NaN
3,4,4,8,"""12055001""","""PS S8W219 SB PSEUDO BREAKER""",055,1,15,0,1,NaN
4,5,5,1,"""01025002""","""PH 25YBSC YARD BOX 2505 SELF CHECK""",025,5,26,0,1,NaN
...,...,...,...,...,...,...,...,...,...,...,...
8459,8460,8460,8,"""120551636""","""PS FL3103 PSUEDO SWITCH""",055,1 2 3 4 5 6 7 8,15,0,1,NaN
8460,8461,8461,8,"""120551637""","""PS FL3T12 FL LOW SD BUS TIE 3T12""",055,1 2 3 4 5 6 7 8,15,0,1,NaN
8461,8462,8462,8,"""120551638""","""PS FL3T21 FL LOW SD BUS TIE 3T21""",055,1 2 3 4 5 6 7 8,15,0,1,NaN
8462,8463,8463,8,"""120551639""","""PS FL3X12 FL XFR BUS TIE 3X12""",055,1 2 3 4 5 6 7 8,15,0,1,NaN


# DATASET Status ready.

# OUTPUT TO DAT FILE :

In [23]:
output_status_name = 'Status88.dat'

In [24]:
with open(output_status_name, 'w') as f:
    f.write('* \n')
    f.write('\t4\tSTATUS\t0\t1\t3\t4\t5\t10\t19\t49\t29\t41\n')
    f.write('*  record OrderNo   Type  Key          Name                                      Stn   AOR               pState   Norm   AlarmGroup   ICAddress\n')

    for index, row in df_status.iterrows():
        f.write("{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\n".format
                ("", row['record'], row['OrderNo'], row['Type'], row['Key'], row['Name'], row['Stn'], row['AOR'], row['pState'], row['Norm'], row['AlarmGroup'], row['ICAddress']))
    f.write(" 0")  

In [25]:
OLD_CODE = """
#Width definition. 
tabspace = "{}/t"
indent_format = "{:<10}"  # Initial TAB
indent2 = "{:1}"
type_format = "{:<6}"
key_format = "{:<8}"
name_format = "{:<37}"
stn_format = "{:<6}"
aor_format = "{:<16}"
pstate_format = "{:<9}"
norm_format = "{:<7}"
alarmgroup_format = "{:<13}"
icaddress_format = "{:<20}"


with open(output_status_name, 'w') as f:
    f.write('*         Type  Key          Name                                      Stn   AOR               pState   Norm   AlarmGroup   ICAddress\n')
    f.write('*         ----  ---          ----                                      ---   ---               ------   ----   ----------   ---------\n')
    f.write('4 STATUS  (1)   (3)          (4)                                       (5)   (10)              (19)     (49)   (29)         (41)\n')

    for i in range(len(df_status)):
        f.write(indent_format.format('') + 
                type_format.format(df_status.loc[i, 'Type']) +
                "\"" + key_format.format(df_status.loc[i, 'Key']) + "\"" +
                indent2.format('') + 
                indent2.format('') + 
                indent2.format('') + 
                "\"" + name_format.format(df_status.loc[i, 'Name']) + "\"" +
                indent2.format('')+
                indent2.format('') + 
                indent2.format('') + 
                stn_format.format(df_status.loc[i, 'Stn']) +
                aor_format.format(df_status.loc[i, 'AOR']) + 
                indent2.format('')+
                indent2.format('') + 
                pstate_format.format(df_status.loc[i, 'pState']) +
                norm_format.format(df_status.loc[i, 'Norm']) +
                alarmgroup_format.format(df_status.loc[i, 'AlarmGroup']) +
                icaddress_format.format(df_status.loc[i, 'ICAddress']) + '\n') """


# END STATUS

-------------------------------

# ANALOG

Description: This script takes in a file named Analog.csv(default) , recognizes its information and interprets the columns. Finally prints and save a Dat format file with the predefined tables for further processing

ANALOG Data conversion REFERENCE :

 Type (1) :  Columns : 
 
                        IF Telem_B  !=  21  . T_ANLG  = type : 1

                        IF Telem_RTU = blank .   C_ANLG =  Type : 2

                        IF NOT Defined. Manual = Type: 3 

                        * CRITERIA: 100% of the Telem_B column is different from 21, while 40% of the Telem_RTU column is blank. To prioritize visibility of Telem_RTU in case of conflict, the priority will be given to C_ANLG (Type 2).

Key (3)   :  XX YYY ZZZ

Name (4)  :  Columns : Name + Desc . 
                If not defined, set to KEY 

Stn (5)   :  ###  -> The first characters of the "Name" column in the current dataset, before the comma (,), reference the value of the KEY column in the Station dataset. This value has a number based on the Order column. This number should be expressed in three digits (e.g., 38 = 038).

AOR (10)  :   Column Zones in Analog.csv

pScale (24): Column EU_HI in Analog.csv

AlarmGrp  (42)  :  Set to default: 1

ICAddress (66)  : It will be declared as NaN.

NominalHiLimits (77,4):  column Alm_unrHi in Analog.csv , named in this df:  Nominal_HiLim

 *Edited:  NominalHiLimits (77:4): RENAMED TO HiLim[1]  (Rsnblty)

NominalLowLimits (78,4):  column Alm_unrLo in Analog.csv  , named in this df: Nominal_LoLim

 *Edited: RENAMED TO LoLim[1]   (Rsnblty)

 ADDED:NominalHiLimits (77:0-3): Column Alm_preHi in Analog.csv, named in this df: HiLim[0]    (High)

 ADDED:NominalLowLimits (78:0-3): Colum Aim_preLo in Analog.csv, named in this df: LoLim[0]    (Low)
 
    
   

In [26]:
#Data CSV Name entry
analog_file = "Analog.csv"

In [27]:
df_analog = pd.read_csv(analog_file)

df_new = pd.DataFrame()
df_new['record'] = range(1, len(df_analog)+1) 
df_new['OrderNo'] = range(1, len(df_analog)+1) 

conditions = [
    (df_analog['Telem_RTU  '] == "  "),
    (df_analog['Telem_B  '] != "21  ")
]
choices = [1, 2]
df_new['Type'] = np.select(conditions, choices, default=3)

df_new['Name'] = df_analog['Name  '].str.replace(',',' ') + " " + df_analog['Desc  ']
df_new['AOR'] = df_analog['Zones  ']
df_new['AlarmGrp'] = 1
df_new['ICAddress'] = "NaN"


#df_new['Nominal_HiLim'] = df_analog['Alm_unrHi  '] #HiLim[0] -> Rsnblty
#df_new['Nominal_LoLim'] = df_analog['Alm_unrLo  '] #LoLim[0] -> Rsnblty

#edited
#----
#df_new['Nominal_HiLim1'] = df_analog['Alm_preHi  '] #HiLim[1] -> High
#df_new['Nominal_LoLim1'] = df_analog['Alm_preLo  '] #LoLim[1] -> Low
############
############



In [28]:

# Convierte a float, convirtiendo errores a NaN
df_copy = df_analog.copy()

# Función para convertir '0  ' o '  ' en 0
def convert_zero(val):
    return 0 if val == '0  ' or val == '  ' else val

# Aplicar la función a todo el DataFrame
df_copy = df_copy.applymap(convert_zero)

# Convierte a float, convirtiendo errores a NaN
df_copy['Alm_unrHi  '] = pd.to_numeric(df_copy['Alm_unrHi  '], errors='coerce')
df_copy['Alm_unrLo  '] = pd.to_numeric(df_copy['Alm_unrLo  '], errors='coerce')
df_copy['Alm_preHi  '] = pd.to_numeric(df_copy['Alm_preHi  '], errors='coerce')
df_copy['Alm_preLo  '] = pd.to_numeric(df_copy['Alm_preLo  '], errors='coerce')

# Luego reemplaza los valores basados en las condiciones
df_copy.loc[df_copy['Alm_unrHi  '] == 0.0, 'Alm_unrHi  '] = 999999.0
df_copy.loc[df_copy['Alm_unrLo  '] == 0.0, 'Alm_unrLo  '] = -999999.0

# Para Alm_preHi y Alm_preLo, necesitamos considerar dos condiciones:
df_copy.loc[(df_copy['Alm_unrLo  '].notna()) & (df_copy['Alm_unrLo  '] != -999999.0), 'Alm_preHi  '] = 999998.0
df_copy.loc[(df_copy['Alm_unrHi  '].notna()) & (df_copy['Alm_unrHi  '] != 999999.0), 'Alm_preLo  '] = -999998.0


df_new['Nominal_HiLim'] = df_copy['Alm_unrHi  '] 
df_new['Nominal_LoLim'] = df_copy['Alm_unrLo  ']
df_new['Nominal_HiLim1'] = df_copy['Alm_preHi  '] 
df_new['Nominal_LoLim1'] = df_copy['Alm_preLo  ']
#----


def keep_decimal_precision(val):
    try:
        float_val = float(val)
        if float_val.is_integer():
            return str(int(float_val))
        else:
            return str(float_val)
    except ValueError:
        return val

df_analog['EU_Hi  '] = df_analog['EU_Hi  '].apply(keep_decimal_precision)
df_new['pScale EU_Hi'] = df_analog['EU_Hi  ']

df_copy['Alm_unrHi  '] = df_copy['Alm_unrHi  '].apply(keep_decimal_precision)
df_copy['Alm_unrLo  '] = df_copy['Alm_unrLo  '].apply(keep_decimal_precision)
df_copy['Alm_preHi  '] = df_copy['Alm_preHi  '].apply(keep_decimal_precision)
df_copy['Alm_preLo  '] = df_copy['Alm_preLo  '].apply(keep_decimal_precision)


df_new['Nominal_HiLim'] = df_copy['Alm_unrHi  '] 
df_new['Nominal_LoLim'] = df_copy['Alm_unrLo  ']
df_new['Nominal_HiLim1'] = df_copy['Alm_preHi  '] 
df_new['Nominal_LoLim1'] = df_copy['Alm_preLo  ']



/var/folders/mj/9v8yq1r92nqcqj3rqh0f2h2h0000gn/T/ipykernel_36481/1720878501.py:9: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_copy = df_copy.applymap(convert_zero)


In [31]:
df_new.head(15)

,record,OrderNo,Type,Name,AOR,AlarmGrp,ICAddress,Nominal_HiLim,Nominal_LoLim,Nominal_HiLim1,Nominal_LoLim1,pScale EU_Hi
0,1,1,2,AB W543A AB W543 A AMPS BKR TRIP,7,1,NaN,470,-30,999998,-999998,1
1,2,2,2,AB W543B AB W543 B AMPS BKR TRIP,7,1,NaN,470,-30,999998,-999998,1
2,3,3,2,AB W543C AB W543 C AMPS BKR TRIP,7,1,NaN,470,-30,999998,-999998,1
3,4,4,2,SG WNDSPD SG WIND SPEED,7,1,NaN,99,-999999,35,-999998,100
4,5,5,1,KH RC CHANNEL ONE RETRY COUNT,1,1,NaN,999999,-999999,0,0,1
5,6,6,1,KH KV1AVG KH TX1 KVA AVERAGE,1,1,NaN,999999,-999999,0,0,1
6,7,7,2,AL P2 ALACHUA TX2 ACCUMULATOR,5,1,NaN,999999,-10,999998,1,3.6234
7,8,8,1,LC 4TOT LC BRK 4 AMP TOTAL,3,1,NaN,999999,-999999,0,0,1
8,9,9,1,SS RC CHANNEL SIX RETRY COUNT,7,1,NaN,999999,-999999,0,0,1
9,10,10,1,ME 1TOT ME BRK 1 AMP TOTAL,1 2 3 4 5 6 7 8,1,NaN,999999,-999999,0,0,1


Reemplazando valores de 0 a su default

In [32]:
df_new['Nominal_HiLim1'] = df_new['Nominal_HiLim1'].replace(0, 999998)
df_new['Nominal_LoLim1'] = df_new['Nominal_LoLim1'].replace(0, -999998)

In [33]:
df_new.head(15)

,record,OrderNo,Type,Name,AOR,AlarmGrp,ICAddress,Nominal_HiLim,Nominal_LoLim,Nominal_HiLim1,Nominal_LoLim1,pScale EU_Hi
0,1,1,2,AB W543A AB W543 A AMPS BKR TRIP,7,1,NaN,470,-30,999998,-999998,1
1,2,2,2,AB W543B AB W543 B AMPS BKR TRIP,7,1,NaN,470,-30,999998,-999998,1
2,3,3,2,AB W543C AB W543 C AMPS BKR TRIP,7,1,NaN,470,-30,999998,-999998,1
3,4,4,2,SG WNDSPD SG WIND SPEED,7,1,NaN,99,-999999,35,-999998,100
4,5,5,1,KH RC CHANNEL ONE RETRY COUNT,1,1,NaN,999999,-999999,0,0,1
5,6,6,1,KH KV1AVG KH TX1 KVA AVERAGE,1,1,NaN,999999,-999999,0,0,1
6,7,7,2,AL P2 ALACHUA TX2 ACCUMULATOR,5,1,NaN,999999,-10,999998,1,3.6234
7,8,8,1,LC 4TOT LC BRK 4 AMP TOTAL,3,1,NaN,999999,-999999,0,0,1
8,9,9,1,SS RC CHANNEL SIX RETRY COUNT,7,1,NaN,999999,-999999,0,0,1
9,10,10,1,ME 1TOT ME BRK 1 AMP TOTAL,1 2 3 4 5 6 7 8,1,NaN,999999,-999999,0,0,1


In [36]:
df_new['Nominal_HiLim1'] = df_new['Nominal_HiLim1'].astype(float)
df_new['Nominal_LoLim1'] = df_new['Nominal_LoLim1'].astype(float)

# Reemplaza los valores
df_new['Nominal_HiLim1'] = df_new['Nominal_HiLim1'].replace(0.0, 999998)
df_new['Nominal_LoLim1'] = df_new['Nominal_LoLim1'].replace(0.0, -999998)

In [38]:
# Redondea los números a enteros y convierte a cadena
df_new['Nominal_HiLim1'] = df_new['Nominal_HiLim1'].astype(int).astype(str)
df_new['Nominal_LoLim1'] = df_new['Nominal_LoLim1'].astype(int).astype(str)




In [39]:
df_new

,record,OrderNo,Type,Name,AOR,AlarmGrp,ICAddress,Nominal_HiLim,Nominal_LoLim,Nominal_HiLim1,Nominal_LoLim1,pScale EU_Hi
0,1,1,2,AB W543A AB W543 A AMPS BKR TRIP,7,1,NaN,470,-30,999998,-999998,1
1,2,2,2,AB W543B AB W543 B AMPS BKR TRIP,7,1,NaN,470,-30,999998,-999998,1
2,3,3,2,AB W543C AB W543 C AMPS BKR TRIP,7,1,NaN,470,-30,999998,-999998,1
3,4,4,2,SG WNDSPD SG WIND SPEED,7,1,NaN,99,-999999,35,-999998,100
4,5,5,1,KH RC CHANNEL ONE RETRY COUNT,1,1,NaN,999999,-999999,999998,-999998,1
...,...,...,...,...,...,...,...,...,...,...,...,...
4405,4406,4406,2,WS TXTEMP WS TX TANK OIL TEMP,5,1,NaN,999999,-999999,999998,-999998,1
4406,4407,4407,2,CA DCV DC VOLTS,5,1,NaN,140,-10,999998,-999998,150
4407,4408,4408,2,WS TXABT WS TX AMBIENT TEMP,5,1,NaN,999999,-999999,999998,-999998,1
4408,4409,4409,2,WS 1KW WS FDR 1 WATTS,5,1,NaN,999999,-999999,999998,-999998,20470


# STN 

In [40]:
df_analog['Key'] = df_analog['Name  '].str.split(',').str[0].str.strip()

stn_values = []

for i in range(len(df_analog)):
    
    matching_row = df_station[df_station['Key'] == df_analog.loc[i, 'Key']]
    
    if not matching_row.empty:
        stn_values.append(f"{matching_row['Order'].values[0]:03}")
    else:
        stn_values.append(np.nan)
df_new['Stn'] = stn_values

In [41]:
df_new

,record,OrderNo,Type,Name,AOR,AlarmGrp,ICAddress,Nominal_HiLim,Nominal_LoLim,Nominal_HiLim1,Nominal_LoLim1,pScale EU_Hi,Stn
0,1,1,2,AB W543A AB W543 A AMPS BKR TRIP,7,1,NaN,470,-30,999998,-999998,1,005
1,2,2,2,AB W543B AB W543 B AMPS BKR TRIP,7,1,NaN,470,-30,999998,-999998,1,005
2,3,3,2,AB W543C AB W543 C AMPS BKR TRIP,7,1,NaN,470,-30,999998,-999998,1,005
3,4,4,2,SG WNDSPD SG WIND SPEED,7,1,NaN,99,-999999,35,-999998,100,008
4,5,5,1,KH RC CHANNEL ONE RETRY COUNT,1,1,NaN,999999,-999999,999998,-999998,1,038
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4405,4406,4406,2,WS TXTEMP WS TX TANK OIL TEMP,5,1,NaN,999999,-999999,999998,-999998,1,003
4406,4407,4407,2,CA DCV DC VOLTS,5,1,NaN,140,-10,999998,-999998,150,010
4407,4408,4408,2,WS TXABT WS TX AMBIENT TEMP,5,1,NaN,999999,-999999,999998,-999998,1,003
4408,4409,4409,2,WS 1KW WS FDR 1 WATTS,5,1,NaN,999999,-999999,999998,-999998,20470,003


# KEY

In [42]:
key_values = []
xx_yyy_counters = {}

In [43]:
for i in range(len(df_new)):
    
    # Asignamos el valor correspondiente a 'XX' según el valor de 'Type'
    if df_new.loc[i, 'Type'] == 1:
        xx = '03'
    elif df_new.loc[i, 'Type'] == 2:
        xx = '04'
    else:
        xx = '99'  # ERROR CASE


    yyy = str(df_new.loc[i, 'Stn'])

    key = xx + yyy

    if key in xx_yyy_counters:
        xx_yyy_counters[key] += 1
    else:
        xx_yyy_counters[key] = 1

    zz = f"{xx_yyy_counters[key]:03}"

    key_values.append(xx + yyy + zz)

df_new['Key'] = key_values

In [44]:
df_new

,record,OrderNo,Type,Name,AOR,AlarmGrp,ICAddress,Nominal_HiLim,Nominal_LoLim,Nominal_HiLim1,Nominal_LoLim1,pScale EU_Hi,Stn,Key
0,1,1,2,AB W543A AB W543 A AMPS BKR TRIP,7,1,NaN,470,-30,999998,-999998,1,005,04005001
1,2,2,2,AB W543B AB W543 B AMPS BKR TRIP,7,1,NaN,470,-30,999998,-999998,1,005,04005002
2,3,3,2,AB W543C AB W543 C AMPS BKR TRIP,7,1,NaN,470,-30,999998,-999998,1,005,04005003
3,4,4,2,SG WNDSPD SG WIND SPEED,7,1,NaN,99,-999999,35,-999998,100,008,04008001
4,5,5,1,KH RC CHANNEL ONE RETRY COUNT,1,1,NaN,999999,-999999,999998,-999998,1,038,03038001
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4405,4406,4406,2,WS TXTEMP WS TX TANK OIL TEMP,5,1,NaN,999999,-999999,999998,-999998,1,003,04003036
4406,4407,4407,2,CA DCV DC VOLTS,5,1,NaN,140,-10,999998,-999998,150,010,04010037
4407,4408,4408,2,WS TXABT WS TX AMBIENT TEMP,5,1,NaN,999999,-999999,999998,-999998,1,003,04003037
4408,4409,4409,2,WS 1KW WS FDR 1 WATTS,5,1,NaN,999999,-999999,999998,-999998,20470,003,04003038


In [45]:
df_new = df_new[['record', 'OrderNo','Type', 'Key', 'Name', 'Stn', 'AOR', 'Nominal_HiLim', 'Nominal_HiLim1', 'Nominal_LoLim', 'Nominal_LoLim1', 'pScale EU_Hi', 'AlarmGrp']].copy()
df_new['ICAddress'] = "NaN"


In [46]:
df_new

,record,OrderNo,Type,Key,Name,Stn,AOR,Nominal_HiLim,Nominal_HiLim1,Nominal_LoLim,Nominal_LoLim1,pScale EU_Hi,AlarmGrp,ICAddress
0,1,1,2,04005001,AB W543A AB W543 A AMPS BKR TRIP,005,7,470,999998,-30,-999998,1,1,NaN
1,2,2,2,04005002,AB W543B AB W543 B AMPS BKR TRIP,005,7,470,999998,-30,-999998,1,1,NaN
2,3,3,2,04005003,AB W543C AB W543 C AMPS BKR TRIP,005,7,470,999998,-30,-999998,1,1,NaN
3,4,4,2,04008001,SG WNDSPD SG WIND SPEED,008,7,99,35,-999999,-999998,100,1,NaN
4,5,5,1,03038001,KH RC CHANNEL ONE RETRY COUNT,038,1,999999,999998,-999999,-999998,1,1,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4405,4406,4406,2,04003036,WS TXTEMP WS TX TANK OIL TEMP,003,5,999999,999998,-999999,-999998,1,1,NaN
4406,4407,4407,2,04010037,CA DCV DC VOLTS,010,5,140,999998,-10,-999998,150,1,NaN
4407,4408,4408,2,04003037,WS TXABT WS TX AMBIENT TEMP,003,5,999999,999998,-999999,-999998,1,1,NaN
4408,4409,4409,2,04003038,WS 1KW WS FDR 1 WATTS,003,5,999999,999998,-999999,-999998,20470,1,NaN


In [47]:
df_new['Key'] = '"'+ df_new['Key'].str.rstrip() + '"'
df_new['Name'] = '"'+ df_new['Name'].str.rstrip() + '"'

In [48]:
df_new

,record,OrderNo,Type,Key,Name,Stn,AOR,Nominal_HiLim,Nominal_HiLim1,Nominal_LoLim,Nominal_LoLim1,pScale EU_Hi,AlarmGrp,ICAddress
0,1,1,2,"""04005001""","""AB W543A AB W543 A AMPS BKR TRIP""",005,7,470,999998,-30,-999998,1,1,NaN
1,2,2,2,"""04005002""","""AB W543B AB W543 B AMPS BKR TRIP""",005,7,470,999998,-30,-999998,1,1,NaN
2,3,3,2,"""04005003""","""AB W543C AB W543 C AMPS BKR TRIP""",005,7,470,999998,-30,-999998,1,1,NaN
3,4,4,2,"""04008001""","""SG WNDSPD SG WIND SPEED""",008,7,99,35,-999999,-999998,100,1,NaN
4,5,5,1,"""03038001""","""KH RC CHANNEL ONE RETRY COUNT""",038,1,999999,999998,-999999,-999998,1,1,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4405,4406,4406,2,"""04003036""","""WS TXTEMP WS TX TANK OIL TEMP""",003,5,999999,999998,-999999,-999998,1,1,NaN
4406,4407,4407,2,"""04010037""","""CA DCV DC VOLTS""",010,5,140,999998,-10,-999998,150,1,NaN
4407,4408,4408,2,"""04003037""","""WS TXABT WS TX AMBIENT TEMP""",003,5,999999,999998,-999999,-999998,1,1,NaN
4408,4409,4409,2,"""04003038""","""WS 1KW WS FDR 1 WATTS""",003,5,999999,999998,-999999,-999998,20470,1,NaN


Output Filename:

In [50]:
output_analog_name = 'Analog88.dat'

In [51]:
with open(output_analog_name, 'w') as f:
    f.write('* \n')
    f.write('\t5\tANALOG\t0\t1\t3\t4\t5\t10\t77,0\t77,4\t78,0\t78,4\t42\t24\t66\n')
    f.write('*         Type  Key          Name                                          Stn   AOR                HiLim[0](High)   HiLim[1](Rsnblty)  LoLim[0](Low)   LoLim[1](Rsnblty)  AlarmGroup   pScale(EU_Hi)  ICAddress\n')

    for index, row in df_new.iterrows():
        f.write("{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\n".format
                ("", row['record'], row['OrderNo'], row['Type'], row['Key'], row['Name'], row['Stn'], row['AOR'], row['Nominal_HiLim'], row['Nominal_HiLim1'], row['Nominal_LoLim'], row['Nominal_LoLim1'], row['pScale EU_Hi'], row['AlarmGrp'], row['ICAddress']))
    f.write(" 0")  

In [52]:
comment= """ indent_format = "{:<10}" 
indent2 = "{:1}"
type_format = "{:<6}"
key_format = "{:<8}"
name_format = "{:<41}"
stn_format = "{:<6}"
aor_format = "{:<17}"
nominal_hilim_format = "{:<16}"
nominal_hilim1_format = "{:<16}"
nominal_lolim_format = "{:<19}"
nominal_lolim1_format = "{:<12}"
alarmgroup_format = "{:<13}"
icaddress_format = "{:<20}"
pscale_format = "{:<15}"
eu_hi_format = "{:<20}"

with open(output_analog_name, 'w') as f:
    f.write('* \n')

    
    for i in range(len(df_new)):
        f.write(indent_format.format('') + 
                type_format.format(str(df_new.loc[i, 'Type'])) +
                "\"" + key_format.format(df_new.loc[i, 'Key']) + "\"" +
                indent2.format('') + 
                indent2.format('') + 
                indent2.format('') + 
                "\"" + name_format.format(df_new.loc[i, 'Name']) + "\"" +
                indent2.format('')+
                indent2.format('') + 
                indent2.format('') + 
                stn_format.format(str(df_new.loc[i, 'Stn'])) +
                aor_format.format(str(df_new.loc[i, 'AOR'])) + 
                indent2.format('')+
                indent2.format('') + 
                nominal_hilim1_format.format(str(df_new.loc[i, 'Nominal_HiLim1']))+
                indent2.format('')+
                nominal_hilim_format.format(str(df_new.loc[i, 'Nominal_HiLim'])) +
                indent2.format('')+
                indent2.format('')+
                indent2.format('')+
                nominal_lolim1_format.format(str(df_new.loc[i, 'Nominal_LoLim1']))+
                
                indent2.format('')+
                indent2.format('')+
                indent2.format('')+
                indent2.format('')+
                nominal_lolim_format.format(str(df_new.loc[i, 'Nominal_LoLim'])) +

                alarmgroup_format.format(str(df_new.loc[i, 'AlarmGrp'])) +
                pscale_format.format(df_new.loc[i, 'pScale EU_Hi']) +
               
                icaddress_format.format(df_new.loc[i, 'ICAddress']) + '\n') """